# Regression Models

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor

## Data Preprocessing

Based on the findings of the EDA stage, I have determined that the following features are the more relevant:

Numeric features:
+ Volume
+ Carat
+ depth and table (in second order)

Categorical features:
+ Color
+ Cut
+ Clarity


Then, I need to cover the following steps:

+ Drop redundant features (x,y,z, density)
+ Preprocess numeric features (volume, carat, depth and table)
+ Preprocess (encode) categorical features (color, cut and Clarity)

In [3]:
filepath = '../datasets/diamonds/diamonds.csv'
# Load the dataset
diamonds = pd.read_csv(filepath)

In [4]:

# Calculate the beta and alpha values
diamonds['beta'] = diamonds['depth'] / 100
diamonds['alpha'] = (1 - diamonds['beta']) * (1 + (diamonds['table'] / 100)**2)

# Calculate the volume of the diamond
diamonds['volume'] = 0.5 * diamonds['z'] * diamonds['x'] * diamonds['y'] * (diamonds['alpha'] + diamonds['beta'])

# Calculate the density of the diamond
diamonds['density'] = diamonds['carat'] / diamonds['volume']

# Drop the auxiliary columns
diamonds.drop(['beta', 'alpha'], axis=1, inplace=True)


In [5]:
# Define the conditions for removing outliers
conditions = [
    (diamonds['carat'] > 0) & (diamonds['price'] < 100),
    (diamonds['z'] > 2) & (diamonds['price'] < 100),
    (diamonds['z'] < 2),
    (diamonds['y'] > 3) & (diamonds['price'] < 100),
    (diamonds['y'] < 2),
    (diamonds['x'] > 2) & (diamonds['price'] < 100),
    (diamonds['x'] < 2),
    (diamonds['table'] > 75),
    (diamonds['depth'] < 50),
    (diamonds['density'] < 0.008)

]

# Create a mask for the rows to be removed
mask = np.any(conditions, axis=0)

# Drop the rows that meet the conditions
diamonds = diamonds[~mask]

# Save the cleaned dataset
#diamonds.to_csv('diamonds_cleaned.csv', index=False)

diamonds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4985 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    4985 non-null   float64
 1   cut      4985 non-null   object 
 2   color    4985 non-null   object 
 3   clarity  4985 non-null   object 
 4   depth    4985 non-null   float64
 5   table    4985 non-null   float64
 6   price    4985 non-null   int64  
 7   x        4985 non-null   float64
 8   y        4985 non-null   float64
 9   z        4985 non-null   float64
 10  volume   4985 non-null   float64
 11  density  4985 non-null   float64
dtypes: float64(8), int64(1), object(3)
memory usage: 506.3+ KB


In [6]:

# Drop redundant features
redundant_features = ['x', 'y', 'z', 'density']
diamonds = diamonds.drop(redundant_features, axis=1)


### Preprocessor

In [7]:
# Extract relevant features
numeric_features = ['volume', 'carat', 'depth', 'table']
categorical_features = ['color', 'cut', 'clarity']
target = 'price'

# Preprocess numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocess categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


## Training the Models

The following are regression models to be considered in this analysis:

1. **Random Forest Regressor:** This model is an ensemble of decision trees and is robust to noisy data. It can handle non-linear relationships well.

2. **Gradient Boosting Regressor:** This is another ensemble method that builds trees sequentially, with each tree correcting the errors of the previous one. It generally performs well on a wide range of datasets.

3. **XGBoost Regressor:** An optimized implementation of gradient boosting that often yields high performance. It's efficient and can handle missing data well.

4. **LASSO Regression (L1 Regularization):** If there are redundant features or multicollinearity, LASSO regression can help by penalizing less important features and shrinking their coefficients to zero.

5. **Elastic Net Regression:** This model combines L1 and L2 regularization, providing a balance between feature selection (like LASSO) and handling correlated features (like Ridge).

At early stages, it's a good idea to try a few different models and compare their performance using cross-validation or a holdout validation set.

In [8]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    diamonds.drop(target, axis=1), diamonds[target], test_size=0.2, random_state=42)


In [9]:
# Preprocessing of training data, train model
preprocessor.fit(X_train, y_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['volume', 'carat', 'depth', 'table']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['color', 'cut', 'clarity'])])

In [10]:
# Preprocess the test data using the preprocessor
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)
# Define models
models = {
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'XGBoost': XGBRegressor(),
    'LASSO Regression':  Lasso(),
    'Elastic Net Regression':  ElasticNet(),
    'Decision Tree': DecisionTreeRegressor()
}

# Evaluate models
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_processed, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    print(f"{name} RMSE: {np.mean(rmse_scores):.4f} (std: {np.std(rmse_scores):.4f})")
    model.fit(X_train_processed, y_train)

# Choose the best model based on cross-validation performance
best_model_name = min(models, key=lambda k: np.mean(np.sqrt(-cross_val_score(models[k], X_train_processed, y_train, cv=5, scoring='neg_mean_squared_error'))))
best_model = models[best_model_name]

# Train the best model on the full training set
best_model.fit(X_train_processed, y_train)

# Evaluate the best model on the test set
test_predictions = best_model.predict(X_test_processed)
test_rmse = np.sqrt(np.mean((test_predictions - y_test)**2))
print(f"\nBest Model ({best_model_name}) Test RMSE: {test_rmse:.4f}")


Random Forest RMSE: 723.5502 (std: 50.6726)
Gradient Boosting RMSE: 804.2004 (std: 51.1857)
XGBoost RMSE: 713.0043 (std: 26.8088)


c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5811527.702595234, tolerance: 4954277.168148588
  model = cd_fast.sparse_enet_coordinate_descent(


LASSO Regression RMSE: 1151.5886 (std: 82.2269)
Elastic Net Regression RMSE: 1645.1323 (std: 107.6607)
Decision Tree RMSE: 948.7411 (std: 48.4257)


c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5811527.702595234, tolerance: 4954277.168148588
  model = cd_fast.sparse_enet_coordinate_descent(



Best Model (XGBoost) Test RMSE: 710.7267


#### Feature Relevance Interpretation

In [16]:
# Considering the XGBoost Regressor model
# Get feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

# Get feature importances from the XGBoost model
feature_importances = best_model.feature_importances_

# Create a dictionary of feature names and importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

# Sort the dictionary by importance scores in descending order
sorted_feature_importance_dict = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))

# Print the feature names and their importance scores
for feature, importance in sorted_feature_importance_dict.items():
    print(f"{feature}: {importance}")

num__carat: 0.30621033906936646
num__volume: 0.151541069149971
cat__clarity_I1: 0.11202339082956314
cat__clarity_SI2: 0.08402672410011292
cat__color_J: 0.04929448291659355
cat__clarity_SI1: 0.042910486459732056
cat__clarity_IF: 0.04141092300415039
cat__color_I: 0.029966726899147034
cat__clarity_VS2: 0.025828417390584946
cat__clarity_VVS2: 0.025498325005173683
cat__clarity_VVS1: 0.024803102016448975
cat__color_F: 0.020394645631313324
cat__color_D: 0.016480518504977226
cat__color_E: 0.01593700796365738
cat__color_H: 0.010113644413650036
cat__clarity_VS1: 0.01010953076183796
cat__cut_Ideal: 0.00948985293507576
cat__color_G: 0.008386184461414814
cat__cut_Fair: 0.00420633889734745
cat__cut_Good: 0.0032914390321820974
num__table: 0.002972440328449011
num__depth: 0.002543751150369644
cat__cut_Premium: 0.0012885506730526686
cat__cut_Very Good: 0.0012721179518848658


In [17]:
# Considering the Decision Tree model
model = models['Decision Tree']
# Get feature importances from the Decision Tree model
feature_importances = model.feature_importances_

# Create a dictionary to store feature names and importances
feature_dict = {feature: importance for feature, importance in zip(feature_names, feature_importances)}

# Sort the dictionary in descending order of importance
sorted_features = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted features and their importances
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")

num__volume: 0.8386714651417257
num__carat: 0.051609069782572736
cat__clarity_SI2: 0.0220569585438036
cat__clarity_SI1: 0.0146388373912231
cat__clarity_I1: 0.014611617713987219
cat__color_J: 0.009582927085472284
cat__color_I: 0.008866868839513483
cat__color_H: 0.006291665137462417
cat__clarity_VS2: 0.006123283799483874
num__depth: 0.004457025496542072
num__table: 0.0037684221511972453
cat__color_D: 0.0037269600769024424
cat__clarity_VS1: 0.003697281736370044
cat__color_G: 0.002803121215613582
cat__color_F: 0.0023231780297708394
cat__color_E: 0.001686868189260598
cat__clarity_VVS2: 0.001197512021636679
cat__clarity_VVS1: 0.0009478618366864614
cat__clarity_IF: 0.0009236202037192895
cat__cut_Ideal: 0.0008612820487972757
cat__cut_Premium: 0.0005715002952561964
cat__cut_Good: 0.00031282853002401764
cat__cut_Very Good: 0.0002429390657518427
cat__cut_Fair: 2.6905667226827253e-05


In [18]:
# Considering the Random Forest model
model = models['Random Forest']
#  Get feature importances from the Decision Tree model
feature_importances = model.feature_importances_

# Create a dictionary to store feature names and importances
feature_dict = {feature: importance for feature, importance in zip(feature_names, feature_importances)}

# Sort the dictionary in descending order of importance
sorted_features = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted features and their importances
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")

num__volume: 0.6618048332950995
num__carat: 0.23049838635182143
cat__clarity_SI2: 0.021473502207410934
cat__clarity_I1: 0.01412474829112116
cat__clarity_SI1: 0.013708894092629863
cat__color_J: 0.008162798199961789
cat__color_I: 0.007296546030283296
cat__color_H: 0.005847683272917857
cat__clarity_VS2: 0.005535479925747223
num__depth: 0.005488864208407825
num__table: 0.00407259324572108
cat__color_D: 0.0037720723986484415
cat__clarity_VS1: 0.003248616160696897
cat__color_G: 0.002549505712697614
cat__color_F: 0.002455079458062683
cat__color_E: 0.002063338473460857
cat__clarity_IF: 0.001980569159509932
cat__cut_Ideal: 0.001571981923334673
cat__clarity_VVS1: 0.001417133154197731
cat__clarity_VVS2: 0.0013246670361226351
cat__cut_Premium: 0.0006269691571145916
cat__cut_Very Good: 0.0003802685070608278
cat__cut_Good: 0.0003356036102564738
cat__cut_Fair: 0.000259866127714643


From the above feature relevance analysis, it can be seen that all models consider that the main aspects to focus on to determine the price of a diamond are:

+ Volume/carat. As it was observed previously
+ Clarity
+ Color

On the other hand, the _depth_, _table_ and _cut_ are not very relevant.

### Hyperparameter Tuning

Here a hyperparameter tuning is run to get the optimal configuration of our models.

In [11]:
### Random Forest Hyperparameter Tuning:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest regressor
rf_model = RandomForestRegressor(random_state=42)

# Create GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_model,
                              param_grid=param_grid_rf,
                              scoring='neg_mean_squared_error',
                              cv=5)
grid_search_rf.fit(X_train_processed, y_train)

# Get the best parameters
best_params_rf = grid_search_rf.best_params_

# Train the model with the best parameters
best_rf_model = RandomForestRegressor(**best_params_rf,
                                      random_state=42)
best_rf_model.fit(X_train_processed, y_train)

### XGBoost Hyperparameter Tuning:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Create an XGBoost regressor
xgb_model = XGBRegressor(random_state=42)

# Create RandomizedSearchCV
random_search_xgb = RandomizedSearchCV(estimator=xgb_model,
                                       param_distributions=param_dist_xgb,
                                       scoring='neg_mean_squared_error',
                                       n_iter=10,
                                       cv=5,
                                       random_state=42)
random_search_xgb.fit(X_train_processed, y_train)

# Get the best parameters
best_params_xgb = random_search_xgb.best_params_
'''
# Train the model with the best parameters
best_xgb_model = XGBRegressor(**best_params_xgb, random_state=42)
train_data = xgb.DMatrix(X_train_processed, label=y_train)

#best_xgb_model.fit(X_train_processed, y_train)
best_xgb_model = xgb.train(**best_params_xgb, train_data, num_boost_round=10)
'''

'\n# Train the model with the best parameters\nbest_xgb_model = XGBRegressor(**best_params_xgb, random_state=42)\ntrain_data = xgb.DMatrix(X_train_processed, label=y_train)\n\n#best_xgb_model.fit(X_train_processed, y_train)\nbest_xgb_model = xgb.train(**best_params_xgb, train_data, num_boost_round=10)\n'

In [18]:
import xgboost as xgb
# Train the model with the best parameters
#best_xgb_model = XGBRegressor(**best_params_xgb, random_state=42)
train_data = xgb.DMatrix(X_train_processed, label=y_train)

#best_xgb_model.fit(X_train_processed, y_train)
best_xgb_model = xgb.train(best_params_xgb, train_data, num_boost_round=1000)

c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:41:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


### Model Evaluation

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predictions on the test set
rf_predictions = best_rf_model.predict(X_test_processed)
test_data = xgb.DMatrix(X_test_processed, label=y_test)
xgb_predictions = best_xgb_model.predict(test_data)

# Evaluate Random Forest model
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = mean_squared_error(y_test, rf_predictions, squared=False)
rf_r2 = r2_score(y_test, rf_predictions)

print("Random Forest Metrics:")
print(f"Mean Absolute Error (MAE): {rf_mae:.2f}")
print(f"Mean Squared Error (MSE): {rf_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rf_rmse:.2f}")
print(f"R-squared (R2): {rf_r2:.2f}")
print()

# Evaluate XGBoost model
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
xgb_r2 = r2_score(y_test, xgb_predictions)

print("XGBoost Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_mae:.2f}")
print(f"Mean Squared Error (MSE): {xgb_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {xgb_rmse:.2f}")
print(f"R-squared (R2): {xgb_r2:.2f}")

Random Forest Metrics:
Mean Absolute Error (MAE): 387.56
Mean Squared Error (MSE): 542776.08
Root Mean Squared Error (RMSE): 736.73
R-squared (R2): 0.97

XGBoost Metrics:
Mean Absolute Error (MAE): 371.32
Mean Squared Error (MSE): 529605.02
Root Mean Squared Error (RMSE): 727.74
R-squared (R2): 0.97


Explanation of Metrics:
- **Mean Absolute Error (MAE):** Represents the average absolute differences between predicted and actual values. Lower values are better, and it is easy to interpret as it is in the same unit as the target variable.

- **Mean Squared Error (MSE):** Represents the average squared differences between predicted and actual values. It penalizes larger errors more than MAE. Lower values are better.

- **Root Mean Squared Error (RMSE):** Represents the square root of the MSE. It provides a measure of the spread of errors in the predictions. Lower values are better.

- **R-squared (R2):** Represents the proportion of the variance in the dependent variable that is predictable from the independent variables. R2 value closer to 1 indicates a better fit. It ranges from 0 to 1, where 1 indicates a perfect fit.

Interpretation:
- Lower values of MAE, MSE, and RMSE indicate better model performance.
- For R-squared, a value close to 1 indicates that the model explains a large proportion of the variability in the target variable.

When interpreting these metrics, it's essential to compare the results to a baseline model or other models to assess relative performance.

### Comparison with Ensemble

In [22]:
# Ensemble predictions by averaging
ensemble_predictions = (rf_predictions + xgb_predictions) / 2

# Evaluate Ensemble model
ensemble_mae = mean_absolute_error(y_test, ensemble_predictions)
ensemble_mse = mean_squared_error(y_test, ensemble_predictions)
ensemble_rmse = mean_squared_error(y_test, ensemble_predictions, squared=False)
ensemble_r2 = r2_score(y_test, ensemble_predictions)

print("Ensemble Metrics:")
print(f"Mean Absolute Error (MAE): {ensemble_mae:.2f}")
print(f"Mean Squared Error (MSE): {ensemble_mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {ensemble_rmse:.2f}")
print(f"R-squared (R2): {ensemble_r2:.2f}")

Ensemble Metrics:
Mean Absolute Error (MAE): 359.68
Mean Squared Error (MSE): 495739.67
Root Mean Squared Error (RMSE): 704.09
R-squared (R2): 0.97


This code calculates the predictions from the ensemble model by taking the average of predictions from the Random Forest and XGBoost models. It then evaluates the ensemble model using the same regression metrics.

Interpretation:
- If the ensemble model performs better than individual models, it suggests that combining the strengths of multiple models contributes to improved predictive performance.
- However, the success of ensemble methods can depend on the diversity and individual performance of the base models.

### Additional Model

In this subsection, I will be taking a step back and train a model which is compatible with Incremental training or partial fit. Then, it is possible to have a model which keeps improving with new data.

In [27]:
from sklearn.linear_model import SGDRegressor

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 0.2, 0.5],
    'penalty': ['l2', 'l1', 'elasticnet', None]
}


# Create a SGD regressor
sgd_model = SGDRegressor()


# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV( estimator=sgd_model,
                            param_grid=param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5)

grid_search.fit(X_train_processed, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
sgd_model = grid_search.best_estimator_
sgd_model.fit(X_train_processed, y_train)

# Evaluate the model on the test set
y_pred = sgd_model.predict(X_test_processed)
mse = mean_squared_error(y_test, y_pred)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Squared Error on Test Set: {mse}')


c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Co

Best Hyperparameters: {'alpha': 0.01, 'eta0': 0.2, 'learning_rate': 'adaptive', 'penalty': None}
Mean Squared Error on Test Set: 1321992.2683089336


c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


However, the model's performance is considerable poor in comparison with the RandomForestRegressor and the XGRegressor. 

**Then, I will take the decision of keeping only those two models and keep training the XGRegressor, which is partial_fi compatible**

## Save Models

In [20]:
import os
import sys
import pickle

def save_object(file_path, obj):
    
    dir_path = os.path.dirname(file_path)

    os.makedirs(dir_path, exist_ok=True)

    with open(file_path, "wb") as file_obj:
        pickle.dump(obj, file_obj)




In [21]:
# Save regressors
filepath = "./models/"
save_object(filepath + f'RandomForestRegressorModel.pkl', best_rf_model)
save_object(filepath + f'XGRegressorModel_v2.pkl', best_xgb_model)

In [10]:
# Save preprocessor
filepath = "./models/"
save_object(filepath + f'preprocessor.pkl', preprocessor)